# Vendée Globe 2020-2021


1. Vitesse moyenne des voiliers
2. Impact du foil sur le classement et la vitesse utile
3. Cartes de la course
4. Impact du foil sur différents paramètres
5. Impact des facteurs sur le classement


&copy; 2024 Francis Wolinski

In [1]:
import pandas as pd
df = (pd
      .read_csv("vg_2020.zip")
      .assign(date=lambda df_: pd.to_datetime(df_.date))
     )

## 1. Vitesse moyenne des voiliers

On calcule la vitesse moyenne de l'ensemble des voiliers par degré de longitude et de latitude.

In [2]:
# Display the average speed by degrees of latitude and longitude
def show_speed(df):
    """Display the average speed by degrees of latitude and longitude"""

    # Display of positions
    def longlat2id(long, lat):
        we = 'W' if long < 0 else 'E'
        sn = 'S' if lat < 0 else 'N'
        return f"{abs(lat)}° {sn}, {abs(long)}° {we}"

    # Build a geojson data structure adapted to the current race for choropleth graph
    def make_feature(long, lat):
        d = dict(type="Feature",
                 properties=dict(position=longlat2id(long, lat)),
                 geometry=dict(type="Polygon", coordinates=[[[long,lat],[long+1,lat],[long+1,lat+1],[long,lat+1],[long,lat]]]))
        return d

    # Compute the average speed by degrees of latitude and longitude
    tab = df[['latitude', 'longitude', 'vitesse_last']].copy()
    tab[['latitude', 'longitude']] = np.floor(tab[['latitude', 'longitude']]).astype(int)

    pivot = (tab
             .pivot_table(index='latitude',
                          columns='longitude',
                          values='vitesse_last')
            )

    tab = pivot.stack().dropna().round(1).reset_index().rename(columns={0:"vitesse"})
    tab["position"] = tab["longitude"].combine(tab["latitude"], longlat2id)

    longs_lats = tab[["longitude", "latitude"]].drop_duplicates().values

    geojson = dict(type="FeatureCollection",
                 features=[make_feature(long, lat) for long, lat in longs_lats])

    fig = px.choropleth_mapbox(data_frame=tab,
                                    geojson=geojson,
                                    locations='position',
                                    color="vitesse",
                                    featureidkey='properties.position',
                                    color_continuous_scale="Bluered",
                                    mapbox_style="carto-positron",
                                    height=600,
                                    width=800,
                                    center={'lat':tab["latitude"].mean(), 'lon':tab["longitude"].mean()},
                                    zoom=0.7)
    fig.update_traces(marker_line_width=0)
    fig.update_layout(title=dict(text="Vitesse moyenne des voiliers"))
    return fig

show_speed(df)

## 3. Cartes de la course

On affiche 3 cartes de la course pour les 11 premiers, les 11 suivants et ainsi de suite.

In [4]:
#vg.show_globe(df, 0, 11)

In [5]:
#vg.show_globe(df, 11, 22)

In [6]:
#vg.show_globe(df, 22, 33)

## 4. Impact du foil sur différents paramètres

On calcule l'impact du foil sur différents paramètres et présenté soit dans le temps soit selon la distance à l'arrivée.

In [7]:
#vg.interact(df)

## 5. Impact des facteurs sur le classement

Pour les facteurs numériques, on calcule par la méthode de SHAP l'impact de leur grandeur sur le classement :
<ul>
    <li>Des facteurs comme les voiliers légers (facteur *Déplacement (poids)* en bleu), les voiliers plus larges (facteur *Largeur* en rouge), ou la présence d'un foil (facteur *foil* en rouge) ont une influence positive sur le classement.</li>
    <li>Inversement, les voiliers plus lourds (facteur *Déplacement (poids)* en rouge), les voiliers plus étroits (facteur *Largeur* en bleu) ou l'absence de foil (facteur *foil* en bleu) ont une influence négative sur le classement.</li>
</ul>
        
*Nota bene* : pour le calcul de l'impact, on a utilisé l'opposé du rang pour obtenir un graphique dans lequel les effets positifs sont à droite et les effets négatifs à gauche.

In [8]:
#vg.show_shap_values(df)